# Generate samples for unimodal systems with standard MCMC

When many prior samples are used with The Joker, and the sampler returns one sample, or the samples returned are within the same mode of the posterior, the posterior pdf is likely unimodal. In these cases, we can use standard MCMC (e.g., Metropolis-Hastings or an ensemble sampler like `emcee`) to generate posterior samples for us. In this example, we will use `emcee` to "continue" sampling for data that are very constraining, so that The Joker only returns a single sample.